#CBDE Lab 4 (part 1) - The Hadoop ecosystem (Hadoop & HBase)

#Assignment

##Setup environment

First, we will download and setup Hadoop and HBase. This notebook is configured to use the following versions of the systems:
*   Java JDK 8
*   Hadoop 3.3.6
*   HBase 2.5.6

In case some of the downloads fail (due to changes in versioning) the download link should be updated as well as all references to the version used.

### Hadoop

Download and install Hadoop

In [ ]:
#Setup JVM 8 (JVM 11 might cause incompatibility issues)
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Download the latest version of Hadoop
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
# Unzip it
# the tar command with the -x flag to extract, -z to uncompress, -v for verbose output, and -f to specify that we are extracting from a file
!tar -xzf hadoop-3.3.6.tar.gz
#copy hadoop file to user/local
!mv hadoop-3.3.6/ /usr/local/

--2023-11-14 21:17:48--  https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz.1’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M   111MB/s    in 5.6s    

2023-11-14 21:17:53 (125 MB/s) - ‘hadoop-3.3.6.tar.gz.1’ saved [730107476/730107476]

mv: cannot move 'hadoop-3.3.6/' to '/usr/local/hadoop-3.3.6': Directory not empty


Set environment variables

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.6/"

In [ ]:
# Add Hadoop BIN to PATH
current_path = os.getenv('PATH')
new_path = current_path+':/usr/local/hadoop-3.3.6/bin/'
os.environ["PATH"] = new_path

### HBase

Download and install HBase

In [ ]:
# Get the latest HBase version
!wget https://dlcdn.apache.org/hbase/2.5.6/hbase-2.5.6-bin.tar.gz
!tar xzf hbase-2.5.6-bin.tar.gz

--2023-11-14 21:19:15--  https://dlcdn.apache.org/hbase/2.5.6/hbase-2.5.6-bin.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314596103 (300M) [application/x-gzip]
Saving to: ‘hbase-2.5.6-bin.tar.gz.1’

hbase-2.5.6-bin.tar 100%[===================>] 300.02M   171MB/s    in 1.8s    

2023-11-14 21:19:17 (171 MB/s) - ‘hbase-2.5.6-bin.tar.gz.1’ saved [314596103/314596103]



Set environment variables

In [ ]:
os.environ["HBASE_HOME"] = "/content/hbase-2.5.6/"
!echo $HBASE_HOME

/content/hbase-2.5.6/


In [ ]:
# Add HBase BIN to PATH
current_path = os.getenv('PATH')
new_path = current_path+':/content/hbase-2.5.6/bin'
os.environ["PATH"] = new_path
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/usr/local/hadoop-3.3.6/bin/:/content/hbase-2.5.6/bin:/usr/local/hadoop-3.3.6/bin/:/content/hbase-2.5.6/bin


### Start HBase server

In [ ]:
!pip install happybase
import happybase
!hbase-daemon.sh start thrift
!start-hbase.sh
!jps

thrift running as process 1860. Stop it first.
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hbase-2.5.6/lib/client-facing-thirdparty/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
master running as process 2191. Stop it first.
1860 ThriftServer
10300 Jps
2191 HMaster


# Dataset description

Both parts of this session consist on processing and analyzing a dataset containing information from the *Census Income Dataset* (https://archive.ics.uci.edu/dataset/2/adult). The schema and characteristics of the dataset are as follows:

1. **age**: continuous.
2. **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
3. **fnlwgt**: continuous.
4. **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
5. **education-num**: continuous.
6. **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
7. **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
8. **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
9. **race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
10. **sex**: Female, Male.
11. **capital-gain**: continuous.
12. **capital-loss**: continuous.
13. **hours-per-week**: continuous.
14. **native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
15. **income**: >50K, <=50K.


In [ ]:
!wget https://archive.ics.uci.edu/static/public/2/adult.zip
!unzip adult.zip

--2023-11-14 21:19:46--  https://archive.ics.uci.edu/static/public/2/adult.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘adult.zip.1’

adult.zip.1             [ <=>                ] 605.70K  --.-KB/s    in 0.1s    

2023-11-14 21:19:46 (4.24 MB/s) - ‘adult.zip.1’ saved [620237]

Archive:  adult.zip
replace Index? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Index                   
replace adult.data? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: adult.data              
  inflating: adult.names             
  inflating: adult.test              
  inflating: old.adult.names         


In [ ]:
import csv, itertools
dataset = []
schema = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']

with open('adult.data') as csvfile:
    for row in itertools.islice(csv.DictReader(csvfile,fieldnames=schema), 100):
      for k,v in row.items():
        row[k] = v.replace(' ', '')
      dataset.append(row)

dataset[0]

{'age': '39',
 'workclass': 'State-gov',
 'fnlwgt': '77516',
 'education': 'Bachelors',
 'education-num': '13',
 'marital-status': 'Never-married',
 'occupation': 'Adm-clerical',
 'relationship': 'Not-in-family',
 'race': 'White',
 'sex': 'Male',
 'capital-gain': '2174',
 'capital-loss': '0',
 'hours-per-week': '40',
 'native-country': 'United-States',
 'income': '<=50K'}

In [ ]:
kxn = happybase.Connection('localhost',9090,autoconnect=False)
kxn

# Exercise 1 - HBase Vertical Partitioning

In this exercise, we are going to focus on the concept of family of HBase, which allows us to vertically fragment instances. Thus, we will programatically define the vertical fragments and access them.

For that, assume a query $Q_1$ that needs to scan the whole table but only fetching attributes *education*, *occupation* and *hours-per-week*.  The equivalent SQL expression for that query would be:

```sql
SELECT education, occupation, hours-per-week
FROM adult
```

## 1.1 Table creation

Provide below the code to create the best table layout (i.e., define the appropriate families) to accomodate such query.

In [ ]:
kxn.open()

# Elimina la taula si existeix
table_name = 'census_data'
try:
    kxn.disable_table(table_name)
    kxn.delete_table(table_name)
    print(f'Taula "{table_name}" desactivada i eliminada amb èxit.')
except Exception as e:

    #Si la taula 'table_name' encara no ha estat creada mai donarà aquesta excepció
    print(f'Error mentre es desactivava/eliminava la taula: {e}')



# Crear taula
kxn.create_table(
    table_name,
    {
        'education': dict(),
        'occupation': dict(),
        'hours-per-week': dict()
    }
)

# Tanca la connexió
kxn.close()

# Mostra les taules per verificar la creació
kxn.open()
print("Taules:", kxn.tables())


# Obté una referència a la taula
kxn.open()
census_table = kxn.table(table_name)
kxn.close()

Taula "census_data" desactivada i eliminada amb èxit.
Taules: [b'census_data', b'census_data_q2']


## 1.2 Populating the table

Provide below the code to populate the table you previously created with the data contained in the **data** array.

In this exercise **you do not need to be concerned** with the choice of key. Thus, you can use an autogenerated timestamp just like the training.

In [ ]:
import time
import random

kxn.open()
b = census_table.batch()
for idx, data_row in enumerate(dataset):
    # Utilitza time.time() com a clau (timestamp)
    key = bytes(str(time.time()), 'UTF-8')

    # Prepara les dades per a la inserció
    data = {
        b'education:': data_row['education'].encode(),
        b'occupation:': data_row['occupation'].encode(),
        b'hours-per-week:': str(data_row['hours-per-week']).encode()
    }

    # Insereix les dades a la taula
    b.put(key, data)

    print(f"Fila {idx + 1} inserida amb èxit.")

    # Dorm per una breu durada (0.1 segon en aquest exemple)
    time.sleep(0.1)

    # Envia l'actualització per lots
    b.send()

kxn.close()

Fila 1 inserida amb èxit.
Fila 2 inserida amb èxit.
Fila 3 inserida amb èxit.
Fila 4 inserida amb èxit.
Fila 5 inserida amb èxit.
Fila 6 inserida amb èxit.
Fila 7 inserida amb èxit.
Fila 8 inserida amb èxit.
Fila 9 inserida amb èxit.
Fila 10 inserida amb èxit.
Fila 11 inserida amb èxit.
Fila 12 inserida amb èxit.
Fila 13 inserida amb èxit.
Fila 14 inserida amb èxit.
Fila 15 inserida amb èxit.
Fila 16 inserida amb èxit.
Fila 17 inserida amb èxit.
Fila 18 inserida amb èxit.
Fila 19 inserida amb èxit.
Fila 20 inserida amb èxit.
Fila 21 inserida amb èxit.
Fila 22 inserida amb èxit.
Fila 23 inserida amb èxit.
Fila 24 inserida amb èxit.
Fila 25 inserida amb èxit.
Fila 26 inserida amb èxit.
Fila 27 inserida amb èxit.
Fila 28 inserida amb èxit.
Fila 29 inserida amb èxit.
Fila 30 inserida amb èxit.
Fila 31 inserida amb èxit.
Fila 32 inserida amb èxit.
Fila 33 inserida amb èxit.
Fila 34 inserida amb èxit.
Fila 35 inserida amb èxit.
Fila 36 inserida amb èxit.
Fila 37 inserida amb èxit.
Fila 38 in

## 1.3 Scanning the table

Provide below the code to implement $Q_1$

In [ ]:
kxn.open()

# Escaneja la taula i imprimeix totes les dades
for key, data in census_table.scan():
    print(key.decode('UTF-8'), data)

# Tanca la connexió
kxn.close()

1699998385.9119892 {b'education:': b'Bachelors', b'hours-per-week:': b'40', b'occupation:': b'Adm-clerical'}
1699998386.027083 {b'education:': b'Bachelors', b'hours-per-week:': b'13', b'occupation:': b'Exec-managerial'}
1699998386.133255 {b'education:': b'HS-grad', b'hours-per-week:': b'40', b'occupation:': b'Handlers-cleaners'}
1699998386.2505035 {b'education:': b'11th', b'hours-per-week:': b'40', b'occupation:': b'Handlers-cleaners'}
1699998386.3564887 {b'education:': b'Bachelors', b'hours-per-week:': b'40', b'occupation:': b'Prof-specialty'}
1699998386.4622467 {b'education:': b'Masters', b'hours-per-week:': b'40', b'occupation:': b'Exec-managerial'}
1699998386.5674853 {b'education:': b'9th', b'hours-per-week:': b'16', b'occupation:': b'Other-service'}
1699998386.672686 {b'education:': b'HS-grad', b'hours-per-week:': b'45', b'occupation:': b'Exec-managerial'}
1699998386.7909224 {b'education:': b'Masters', b'hours-per-week:': b'50', b'occupation:': b'Prof-specialty'}
1699998386.896329

# Exercise 2 - HBase Key Design

This exercise asks you to design the key of the table to benefit from the distributed B-tree of HBase and avoid table scans when doing specific searches. Thus, assume a query $Q_2$ that retrieves all attributes where native-country = 'United-States' and occupation = 'Tech-support' = 0. The equivalent SQL expression for that query would be:

```sql
SELECT *
FROM adult
WHERE native-country = 'United-States' AND occupation = 'Tech-support'
```

For this exercise, you can recreate the table from the **data** array with a single column family (provide the code below).

In [ ]:
kxn.open()

# Elimina la taula si existeix
table_name = 'census_data_q2'
try:
    kxn.disable_table(table_name)
    kxn.delete_table(table_name)
    print(f'Taula "{table_name}" desactivada i eliminada amb èxit.')
except Exception as e:

    #Si la taula 'table_name' encara no ha estat creada mai donarà aquesta excepció
    print(f'Error mentre es desactivava/eliminava la taula: {e}')

# Crea la taula amb una única família de columnes
try:
    kxn.create_table(
        table_name,
        {
            'data': dict()
        }
    )
    print(f'Taula "{table_name}" creada amb èxit amb una única família de columnes.')
except Exception as e:
    print(f'Error mentre es creava la taula: {e}')


# Obté una referència a la taula
kxn.open()
census_table_q2 = kxn.table(table_name)
kxn.close()

Taula "census_data_q2" desactivada i eliminada amb èxit.
Taula "census_data_q2" creada amb èxit amb una única família de columnes.


## 2.1 Key creation

Provide below the code to populate the table so that it generates the most appropriate key design to efficiently answer $Q_2$.

In [ ]:
kxn.open()

b = census_table_q2.batch()
for idx, data_row in enumerate(dataset):
    # Dissenyar la clau per respondre eficientment a la Q2: "<native-country>_<occupation>_<timestamp>"
    key = f"{data_row['native-country']}_{data_row['occupation']}_{int(time.time() * 1000)}"

    # Prepara les dades per a la inserció
    data = {
        b'data:age': data_row['age'].encode(),
        b'data:workclass': data_row['workclass'].encode(),
        b'data:fnlwgt': data_row['fnlwgt'].encode(),
        # Inclou altres columnes segons sigui necessari
    }

    # Insereix les dades a la taula
    b.put(key.encode(), data)

    print(f"Fila {idx + 1} inserida amb èxit amb la clau: {key}")

    # Dorm per una breu durada (0.1 segon en aquest exemple)
    time.sleep(0.1)

# Envia l'actualització per lots
b.send()

kxn.close()

Fila 1 inserida amb èxit amb la clau: United-States_Adm-clerical_1699998411975
Fila 2 inserida amb èxit amb la clau: United-States_Exec-managerial_1699998412076
Fila 3 inserida amb èxit amb la clau: United-States_Handlers-cleaners_1699998412177
Fila 4 inserida amb èxit amb la clau: United-States_Handlers-cleaners_1699998412277
Fila 5 inserida amb èxit amb la clau: Cuba_Prof-specialty_1699998412378
Fila 6 inserida amb èxit amb la clau: United-States_Exec-managerial_1699998412478
Fila 7 inserida amb èxit amb la clau: Jamaica_Other-service_1699998412578
Fila 8 inserida amb èxit amb la clau: United-States_Exec-managerial_1699998412679
Fila 9 inserida amb èxit amb la clau: United-States_Prof-specialty_1699998412779
Fila 10 inserida amb èxit amb la clau: United-States_Exec-managerial_1699998412879
Fila 11 inserida amb èxit amb la clau: United-States_Exec-managerial_1699998412980
Fila 12 inserida amb èxit amb la clau: India_Prof-specialty_1699998413080
Fila 13 inserida amb èxit amb la clau: U

## 2.2 Range queries

Provide below the code required to efficiently implement $Q_2$

In [ ]:
kxn.open()

# Defineix l'interval d'escaneig per a la Q2
start_key = 'United-States_Tech-support_'
end_key = 'United-States_Tech-support_\xff'  # '\xff' s'utilitza com a final de l'interval per cobrir totes les possibles marques de temps

# Escaneja la taula dins de l'interval especificat
scanner = census_table_q2.scan(row_start=start_key, row_stop=end_key)

# Imprimeix els resultats
for key, data in scanner:
        print(f"Key: {key.decode('UTF-8')}, Data: {data}")

kxn.close()

Key: United-States_Tech-support_1699998414386, Data: {b'data:age': b'59', b'data:fnlwgt': b'109015', b'data:workclass': b'Private'}
Key: United-States_Tech-support_1699998414486, Data: {b'data:age': b'56', b'data:fnlwgt': b'216851', b'data:workclass': b'Local-gov'}
Key: United-States_Tech-support_1699998416193, Data: {b'data:age': b'24', b'data:fnlwgt': b'172987', b'data:workclass': b'Private'}
Key: United-States_Tech-support_1699998417497, Data: {b'data:age': b'43', b'data:fnlwgt': b'237993', b'data:workclass': b'Private'}
Key: United-States_Tech-support_1699998418401, Data: {b'data:age': b'29', b'data:fnlwgt': b'105598', b'data:workclass': b'Private'}
